# Cuaderno de Notas de Métodos Numéricos
## Interpolación: datos reales y errores.
### Prof. Jorge I. Zuluaga


#### Ejecutarme siempre

A continuación descargamos en el espacio virtual de este cuaderno todos los archivos necesarios para que las notas del curso funcionen bien:

In [ ]:
!git clone https://github.com/JorgeZuluaga/NotasMetodosNumericos
!ln -s NotasMetodosNumericos mn
!make -C mn pull

Cloning into 'NotasMetodosNumericos'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 108 (delta 46), reused 82 (delta 20), pack-reused 0
Receiving objects: 100% (108/108), 8.53 MiB | 19.32 MiB/s, done.
Resolving deltas: 100% (46/46), done.
make: Entering directory '/content/NotasMetodosNumericos'
git reset --hard HEAD
HEAD is now at 876de31 Commit
git pull origin master
From https://github.com/JorgeZuluaga/NotasMetodosNumericos
 * branch            master     -> FETCH_HEAD
Already up to date.
make: Leaving directory '/content/NotasMetodosNumericos'


## Un vistazo de la clase

  Qué veremos en esta clase:

  - Diferencias divididas
  - Interpolación de Hermite

  Para leer en el libro: sección 3.3 "Diferencias divididas" y sección 3.4 "Interpolación de Hermite"

## Diferencias divididas

- El método de las diferencias divididas aparece cuando queremos escribir el Polinomio interpolante en la forma:

  $$
P_{n}(x)=a_{0}+a_{1}\left(x-x_{0}\right)+a_{2}\left(x-x_{0}\right)\left(x-x_{1}\right)+\cdots+a_{n}\left(x-x_{0}\right) \cdots\left(x-x_{n-1}\right)
$$

- Nótese que por la condición de que $P_n(x_i)=f(x_i)$ entonces:
  
  $$
  a_0=f(x_0)
  $$
  y 
  $$
  a_{1}=\frac{f\left(x_{1}\right)-f\left(x_{0}\right)}{x_{1}-x_{0}}
  $$

- Si definimos:

  $$
  f\left[x_{i}\right]=f\left(x_{i}\right)
  $$
  
  y 

  $$
  f\left[x_{i}, x_{i+1}\right]=\frac{f\left[x_{i+1}\right]-f\left[x_{i}\right]}{x_{i+1}-x_{i}}
  $$

  entonces:

  $$
  a_0=f[x_0]
  $$

  y 

  $$
  a_1=f[x_0,x_1]
  $$

- Llamamos a $f[x_i]$ y a $f[x_i,x_{i+1}]$ diferencias divididas y en este caso se llaman las diferencias 0-esima y primera, respectivamente.

- La segunda diferencia dividida se define como:

  $$
  f\left[x_{i}, x_{i+1}, x_{i+2}\right]=\frac{f\left[x_{i+1}, x_{i+2}\right]-f\left[x_{i}, x_{i+1}\right]}{x_{i+2}-x_{i}}
  $$

- Se puede generalizar para producir la $k-$esima diferencia dividida así:

  $$
  f\left[x_{i}, x_{i+1}, \ldots, x_{i+k-1}, x_{i+k}\right]=\frac{f\left[x_{i+1}, x_{i+2}, \ldots, x_{i+k}\right]-f\left[x_{i}, x_{i+1}, \ldots, x_{i+k-1}\right]}{x_{i+k}-x_{i}}
  $$

- Con estas definiciones es posible probar que el polinomio interpolante de Lagrange es:

  $$
  \begin{aligned}
P_{n}(x)=& f\left[x_{0}\right]+f\left[x_{0}, x_{1}\right]\left(x-x_{0}\right)+a_{2}\left(x-x_{0}\right)\left(x-x_{1}\right) \\
&+\cdots+a_{n}\left(x-x_{0}\right)\left(x-x_{1}\right) \cdots\left(x-x_{n-1}\right)
\end{aligned}
  $$

- Generalizando queda:

  $$
  P_{n}(x)=f\left[x_{0}\right]+\sum_{k=1}^{n} f\left[x_{0}, x_{1}, \ldots, x_{k}\right]\left(x-x_{0}\right) \cdots\left(x-x_{k-1}\right)
  $$

- Las diferencias divididas se pueden calcular usando un esquema tabular como se muestra en la figura:

 <center>
  <img src="https://raw.githubusercontent.com/JorgeZuluaga/NotasMetodosNumericos/master/figuras/diferencias-divididas.png" width=800>
  </center>

### Algoritmo de diferencias divididas

- En la imagen abajo se muestra el algoritmo para calcular los elementos de la tabla de diferencias divididas: 

  <center>
  <img src="https://raw.githubusercontent.com/JorgeZuluaga/NotasMetodosNumericos/master/figuras/algoritmo-diferencias-divididas.png" width=600>
  </center>

- Una implementación en Python de este algoritmo se muestra abajo:


## Interpolación de Hermite

- La interpolación de Hermite busca resolver el problema que habíamos visto al interpolar datos reales, en los que la función oscilaba de forma "salvaje".

- La interpolación de Hermite es un método para encontrar un polinomio que pase por los puntos de un conjunto de datos pero que también tenga primera derivada igual a la derivada del polinomio.

- El polinomio que cumple esa condición se llama polinomio de Hermite y es de grado $2n+1$:

  $$
  H_{2 n+1}(x)=\sum_{j=0}^{n} f\left(x_{j}\right) H_{n, j}(x)+\sum_{j=0}^{n} f^{\prime}\left(x_{j}\right) \hat{H}_{n, j}(x)
  $$

- Los coeficientes del polinomio se calculan así:

  $$
  H_{n, j}(x)=\left[1-2\left(x-x_{j}\right) L_{n, j}^{\prime}\left(x_{j}\right)\right] L_{n, j}^{2}(x)
  $$

  y 

  $$
  \hat{H}_{n, j}(x)=\left(x-x_{j}\right) L_{n, j}^{2}(x)
  $$

- Como el cálculo derivando los coeficientes de los polinomios de Lagrange es tan laborioso, existe una manera de calcular los polinomios de Hermite usando diferencias divididas:

  $$
  H_{2 n+1}(x)=f\left[z_{0}\right]+\sum_{k=1}^{2 n+1} f\left[z_{0}, \ldots, z_{k}\right]\left(x-z_{0}\right)\left(x-z_{1}\right) \cdots\left(x-z_{k-1}\right)
  $$

  donde introducimos un conjunto de vaiables nuevas $z_{2 i}=z_{2 i+1}=x_{i}$

- El método se ilustra en la tabla a continuación:



### Algoritmo de la interpolación de Hermite

- Este es el algoritmo de la interpolación de Hermite:

- Esta es la implementación en Python del método:




## Continuará...

## Ejercicios

- Resolver los ejercicios de la sección 3.1:

  - Ejercicio 7, literal a.
  - Ejercicio 14, literal a.

-----
*Fin*